## Notes

This illustrates solution of the spheroidal active particle trajectory models using scipy's solve_ivp.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.integrate import solve_ivp

In [ ]:
# Define the square duct Poiseuille flow and its derivatives (using the series solution)
def w(y,N=10,U=10.0):
    M = 1.0
    W = 1.0-y[1]**2
    sign = +1
    for n in range(N):
        kn = (2*n+1)*np.pi/2
        sign *= -1
        temp = 4*sign/(kn**3*np.cosh(kn))
        W += temp*np.cosh(kn*y[0])*np.cos(kn*y[1])
        M += temp
    return W*U/M
def dwdx(y,N=10,U=10.0):
    M = 1.0
    W = 0*y[0] # ensure correct shape...
    sign = +1
    for n in range(N):
        kn = (2*n+1)*np.pi/2
        sign *= -1
        temp = 4*sign/(kn**3*np.cosh(kn))
        W += kn*temp*np.sinh(kn*y[0])*np.cos(kn*y[1])
        M += temp
    return W*U/M
def dwdy(y,N=10,U=10.0):
    M = 1.0
    W = -2*y[1]
    sign = +1
    for n in range(N):
        kn = (2*n+1)*np.pi/2
        sign *= -1
        temp = 4*sign/(kn**3*np.cosh(kn))
        W += -kn*temp*np.cosh(kn*y[0])*np.sin(kn*y[1])
        M += temp
    return W*U/M

# Define ODE functions for each case
def ode_s(t,y,Cs,N,U):
    ez = Cs+0.5*w(y,N,U)
    return np.array([y[2],
                     y[3],
                     -0.5*ez*dwdx(y,N,U),
                     -0.5*ez*dwdy(y,N,U)])
def ode_p(t,y,Cp,N,U):
    fG = (2*G/(1+G))**0.5
    w_ = w(y,N,U)
    Fw = np.cosh(Cp+0.5*(1+G)*fG*w_)
    ez = np.tanh(Cp+0.5*(1+G)*fG*w_)/fG
    return np.array([y[2]/Fw,
                     y[3]/Fw,
                     -0.5*(1-G)*dwdx(y,N,U)*ez*Fw,
                     -0.5*(1-G)*dwdy(y,N,U)*ez*Fw])
def ode_o(t,y,Co,N,U):
    fG = (-2*G/(1+G))**0.5
    w_ = w(y,N,U)
    Fw = np.cos(Co+0.5*(1+G)*fG*w_)
    ez = np.tan(Co+0.5*(1+G)*fG*w_)/fG
    return np.array([y[2]/Fw,
                     y[3]/Fw,
                     -0.5*(1-G)*dwdx(y,N,U)*ez*Fw,
                     -0.5*(1-G)*dwdy(y,N,U)*ez*Fw])

# A convenience function for plotting solutions
def plot_ode_solution(xs,ys,exs,eys,ezs):
    fig = plt.figure(figsize=(9,2.5))
    ax = fig.add_subplot(131)
    ax.plot(ts,xs,label='$x$',lw=1)
    ax.plot(ts,ys,label='$y$',lw=1)
    ax.set_xlabel('$t$')
    ax.legend(loc='upper right')
    ax = fig.add_subplot(132)
    ax.plot(xs,ys,lw=1)
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_aspect(1.0)
    ax = fig.add_subplot(133,projection='3d')
    ax.plot(exs,eys,ezs,lw=1)
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    ax.set_zlim(-1,1)
    ax.set_xlabel('$e_x$')
    ax.set_ylabel('$e_y$')
    ax.set_zlabel('$e_z$')
    plt.tight_layout()
    plt.show()

In [ ]:
# Start with solving the spherical case

# Set parameters
U = 10.0
N = 10
dt = 0.5**6
nt = 2**(8+6)
y0 = [0.2,0.8]
dy0 = [0.0,0.0] # equivalent to ex(0),ey(0)
ez0 = -1.0
assert np.isclose(1-dy0[0]**2-dy0[1]**2,ez0**2)
Cs = ez0-0.5*w(y0,N,U)

# Specify time stepping and solve
ts = np.linspace(0,nt*dt,nt+1)
solution = solve_ivp(ode_s,[ts[0],ts[-1]],y0+dy0,t_eval=ts,
                     method='DOP853',rtol=1.0E-9,atol=1.0E-12,
                     args=(Cs,N,U),)

# Plot the solution
xs = solution.y[0]
ys = solution.y[1]
exs = solution.y[2]
eys = solution.y[3]
ezs = Cs+0.5*w([xs,ys],N,U)
plot_ode_solution(xs,ys,exs,eys,ezs)

# Plot the Hamiltonian error (rather the error in ||e||^2)
H = solution.y[2]**2+solution.y[3]**2+ezs**2-1
plt.figure(figsize=(4,3))
plt.plot(ts,H,lw=1)
plt.show()

In [ ]:
# Now solve the prolate case

# Set parameters
U = 10.0
N = 10
G = 0.6 # gamma=2
dt = 0.5**6
nt = 2**(8+6)
y0 = [0.2,0.8]
dy0 = [0.0,0.0] # equivalent to ex(0),ey(0)
ez0 = -1.0
assert np.isclose(1-dy0[0]**2-dy0[1]**2,ez0**2)
fG = (2*G/(1+G))**0.5
Cp = np.arctanh(fG*ez0)-0.5*(1+G)*fG*w(y0,N,U)

# Specify time stepping and solve
ts = np.linspace(0,nt*dt,nt+1)
solution = solve_ivp(ode_p,[ts[0],ts[-1]],y0+dy0,t_eval=ts,
                     method='DOP853',rtol=1.0E-9,atol=1.0E-12,
                     args=(Cp,N,U),)

# Plot the solution
xs = solution.y[0]
ys = solution.y[1]
ws = w([xs,ys],N,U) 
Fw = np.cosh(Cp+0.5*(1+G)*fG*ws)
exs = solution.y[2]/Fw
eys = solution.y[3]/Fw
ezs = np.tanh(Cp+0.5*(1+G)*fG*ws)/fG
plot_ode_solution(xs,ys,exs,eys,ezs)

# Plot the Hamiltonian error (rather the error in ||e||^2)
H = exs**2+eys**2+ezs**2-1
plt.figure(figsize=(4,3))
plt.plot(ts,H,lw=1)
plt.show()

In [ ]:
# Now solve the oblate case

# Set parameters
U = 10.0
N = 10
G = -0.6 # gamma=1/2
dt = 0.5**6
nt = 2**(8+6)
y0 = [0.2,0.8]
dy0 = [0.0,0.0] # equivalent to ex(0),ey(0)
ez0 = -1.0
assert np.isclose(1-dy0[0]**2-dy0[1]**2,ez0**2)
fG = (-2*G/(1+G))**0.5
Co = np.arctan(fG*ez0)-0.5*(1+G)*fG*w(y0,N,U)

# Specify time stepping and solve
ts = np.linspace(0,nt*dt,nt+1)
solution = solve_ivp(ode_o,[ts[0],ts[-1]],y0+dy0,t_eval=ts,
                     method='DOP853',rtol=1.0E-9,atol=1.0E-12,
                     args=(Co,N,U),)

# Plot the solution
xs = solution.y[0]
ys = solution.y[1]
ws = w([xs,ys],N,U) 
Fw = np.cos(Co+0.5*(1+G)*fG*ws)
exs = solution.y[2]/Fw
eys = solution.y[3]/Fw
ezs = np.tan(Co+0.5*(1+G)*fG*ws)/fG
plot_ode_solution(xs,ys,exs,eys,ezs)

# Plot the Hamiltonian error (rather the error in ||e||^2)
H = exs**2+eys**2+ezs**2-1
plt.figure(figsize=(4,3))
plt.plot(ts,H,lw=1)
plt.show()